<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0614sentence_bert_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- オリジナル: https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens


In [ ]:
import os
import sys
import numpy as np
import unicodedata
from termcolor import colored

# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストールする
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install transformers > /dev/null 2>&1 

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1
    !pip install jaconv > /dev/null 2>&1 
    !pip install japanize_matplotlib > /dev/null 2>&1

In [ ]:
from transformers import BertJapaneseTokenizer, BertModel
import torch


class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)


MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens"
model = SentenceBertJapanese(MODEL_NAME)

sentences = ["暴走したAI", "暴走した人工知能"]
sentence_embeddings = model.encode(sentences, batch_size=8)

#print("Sentence embeddings:", sentence_embeddings)


In [ ]:
import numpy as np
import scipy

# メモ 以下では，sentences という list に複数の日本語文をカンマで区切って入力し, model.encode(sentences) の
# ようにして呼び出す。戻り値は，各文に対応した文ベクトル。
# この戻ってきた文ベクトルに対して，以下では，ベクトルの内積を書くベクトルの 2乗和を開平で割って
# コサイン類似度を求めている。
# 2022_0614 近藤先生との zoom ミーティング時に検討した。

sentences = ["暴走したAI", "暴走した人工知能"]
#sentences = ["AI", "人工知能"]
sentences = ["ピカピカと雷と光る", "ゴロゴロと雷が鳴る"]
sentences = ["ピカピカと雷と光る", "ピカピカとウィンカー"]
sentences = ["ピカピカと雷と光る", "ピカピカとウィンカーが光る"]
sentences = ['夏の暑さが肌にじりじり感じる', '真夏の太陽がさんさんと照り付ける']
sentences = ['夏の暑さが肌にじりじり感じる', '真夏の太陽がさんさんと照り付ける']
sentences = ['夏の暑さが肌にじりじり感じる', '真夏の太陽がさんさんと照り付ける']
#sentences = ['教授は国際会議で発表した。', '教授の国際会議での発表は失敗だった。']  # 0.517
#sentences = ['教授の国際会議への投稿はリジェクトされた。', '教授の国際会議での発表は失敗だった。']  # 0.703
#sentences = ['教授の国際会議への投稿はリジェクトされた。', '教授の国際会議での発表は捏造だった。'] # 0.575
#sentences = ['彼の国際ジャーナル論文はリジェクトされた。', '教授の国際会議での発表は捏造だった。'] # 0.365
#sentences = ['彼の国際ジャーナル論文は取り消された。', '教授の国際会議での発表は捏造だった。']    # 0.428
#sentences = ['彼の国際ジャーナル論文は取り消された。', '教授は国際会議で捏造データを発表した。']   # 0.421
sentences = ['彼の国際ジャーナル論文は取り消された。教授は国際会議で新たなデータを発表した。',
'教授の国際ジャーナル論文は取り消された。彼は国際会議で捏造データを発表した。']   # 0.888
sentences = ['今日は夕立が来そうだ。ほら，雷がピカピカ光った。',
'今日は夕立が来そうだ。ほら，空からゴロゴロと音が聞こえた。']   # 0.785
sentences = ['今日は朝からどんより曇ってる。ピカピカ光った。',
'今日は夕立が来そうだ。ほら，空からゴロゴロと音が聞こえた。']   # 0.540
sentences = ['今日は朝からどんより曇ってる。ピカピカ光った。',
'夕立が来そうだね。ほら，空からゴロゴロと音が聞こえたよ。']   # 0.503
sentences = ['ピカピカ光った。','空からゴロゴロと音が聞こえたよ。']      # 0.325
# sentences = ['空がピカピカ光った。','空からゴロゴロと音が聞こえたよ。']   # 0.530
# sentences = ['雲がピカピカ光った。','空からゴロゴロと音が聞こえたよ。']   # 0.438
# sentences = ['雲がピカピカ光った。','空からゴロゴロと鳴った。']          # 0.356
# sentences = ['ピカピカ','ゴロゴロ']                                 # 0.108
# sentences = ['雲がピカピカと光った。','空がゴロゴロと鳴った。']          # 0.357
# sentences = ['雲が光る。','空が鳴る。']                              # 0.497
# sentences = ['雲。','空。']                                        # 0.395
# sentences = ['雲。','雷。']                                        # 0.578
sentence_embeddings = model.encode(sentences, batch_size=8)

s0 = sentence_embeddings[0].detach().numpy()
s1 = sentence_embeddings[1].detach().numpy()
s0norm = np.sqrt((s0 ** 2).sum())
s1norm = np.sqrt((s1 ** 2).sum())
print(np.dot(s0,s1) / (s0norm * s1norm))


In [ ]:
# 入力文をトークナイズするには，以下のようにする。
# 結果は，'input_ids', 'token_type_ids', 'attention_mask' という 3 項目からなる辞書が返る。
model.tokenizer(sentences)

# 上を整列して印字したければ，下記のようにする
#for k, v in model.tokenizer(sentences).items():
#    print(k, v)


In [ ]:
# 特殊トークンはモデルごとに トークン番号が異なるので，念の為表示させる
model.tokenizer.special_tokens_map

In [ ]:
# 2 つ上のセルで，戻ってきたトークン番号を，実際の単語に変換するには `.convert_ids_to_tokens()` 関数を用いる
#model.tokenizer.convert_tokens_to_ids('[CLS]')
print(model.tokenizer.convert_ids_to_tokens([2, 14739, 27056, 948, 28468, 10, 8, 3]))
print(model.tokenizer.convert_ids_to_tokens([2, 454, 40, 18968, 28796, 28505, 13, 419, 14, 15440, 10, 54, 8, 3]))